In [55]:
import pandas as pd

import pickle
from pathlib import Path

In [46]:
all_months = ["2023-05-01_2023-05-31", "2023-06-01_2023-06-30", "2023-07-01_2023-07-31", "2023-08-01_2023-08-31", "2023-09-01_2023-09-30",
    "2023-10-01_2023-10-31", "2023-11-01_2023-11-30", "2023-12-01_2023-12-31", "2024-01-01_2024-01-31", "2024-02-01_2024-02-29",
    "2024-03-01_2024-03-31", "2024-04-01_2024-04-30"]

base_img_dir = Path("/projects/frame_align/data/img_data/")
month_dir_paths = [base_img_dir / month for month in all_months]
sorted_month_dir_paths = sorted(month_dir_paths)


# there is *failed_urls.json and *failed_ids.json in month dir. delete them
def clean_failed_files(month_dir):
    failed_urls = month_dir / "failed_urls.json"
    failed_ids = month_dir / "failed_ids.json"

    if failed_urls.exists():
        print(f"Deleting {failed_urls}")
        failed_urls.unlink()

    if failed_ids.exists():
        print(f"Deleting {failed_ids}")
        failed_ids.unlink()

for month_dir in sorted_month_dir_paths:
    clean_failed_files(month_dir)

Make path of monthly directories

In [56]:
all_months = ["2023-05-01_2023-05-31", "2023-06-01_2023-06-30", "2023-07-01_2023-07-31", "2023-08-01_2023-08-31", "2023-09-01_2023-09-30",
    "2023-10-01_2023-10-31", "2023-11-01_2023-11-30", "2023-12-01_2023-12-31", "2024-01-01_2024-01-31", "2024-02-01_2024-02-29",
    "2024-03-01_2024-03-31", "2024-04-01_2024-04-30"]

base_img_dir = Path("/projects/frame_align/data/news-img-data/")
month_dir_paths = [base_img_dir / month for month in all_months]
sorted_month_dir_paths = sorted(month_dir_paths)

print("month_dir_paths", sorted_month_dir_paths)

month_dir_paths [PosixPath('/projects/frame_align/data/news-img-data/2023-05-01_2023-05-31'), PosixPath('/projects/frame_align/data/news-img-data/2023-06-01_2023-06-30'), PosixPath('/projects/frame_align/data/news-img-data/2023-07-01_2023-07-31'), PosixPath('/projects/frame_align/data/news-img-data/2023-08-01_2023-08-31'), PosixPath('/projects/frame_align/data/news-img-data/2023-09-01_2023-09-30'), PosixPath('/projects/frame_align/data/news-img-data/2023-10-01_2023-10-31'), PosixPath('/projects/frame_align/data/news-img-data/2023-11-01_2023-11-30'), PosixPath('/projects/frame_align/data/news-img-data/2023-12-01_2023-12-31'), PosixPath('/projects/frame_align/data/news-img-data/2024-01-01_2024-01-31'), PosixPath('/projects/frame_align/data/news-img-data/2024-02-01_2024-02-29'), PosixPath('/projects/frame_align/data/news-img-data/2024-03-01_2024-03-31'), PosixPath('/projects/frame_align/data/news-img-data/2024-04-01_2024-04-30')]


In [27]:
# csv_path_base = Path("/projects/frame_align/data/raw/2023-2024")

# # look for datawithtopiclabels.csv in csv_path_base/month
# csv_dir_paths = [csv_path_base / month / "datawithtopiclabels.csv" for month in all_months]
# sorted_csv_dir_paths = sorted(csv_dir_paths)

# print("csv_dir_paths", sorted_csv_dir_paths)

# # count length of each csv
# csv_len = []
# for csv_dir in sorted_csv_dir_paths:
#     df = pd.read_csv(csv_dir)
#     csv_len.append(len(df))

# print("csv_len", csv_len)


In [57]:
sorted_month_dir_paths

[PosixPath('/projects/frame_align/data/news-img-data/2023-05-01_2023-05-31'),
 PosixPath('/projects/frame_align/data/news-img-data/2023-06-01_2023-06-30'),
 PosixPath('/projects/frame_align/data/news-img-data/2023-07-01_2023-07-31'),
 PosixPath('/projects/frame_align/data/news-img-data/2023-08-01_2023-08-31'),
 PosixPath('/projects/frame_align/data/news-img-data/2023-09-01_2023-09-30'),
 PosixPath('/projects/frame_align/data/news-img-data/2023-10-01_2023-10-31'),
 PosixPath('/projects/frame_align/data/news-img-data/2023-11-01_2023-11-30'),
 PosixPath('/projects/frame_align/data/news-img-data/2023-12-01_2023-12-31'),
 PosixPath('/projects/frame_align/data/news-img-data/2024-01-01_2024-01-31'),
 PosixPath('/projects/frame_align/data/news-img-data/2024-02-01_2024-02-29'),
 PosixPath('/projects/frame_align/data/news-img-data/2024-03-01_2024-03-31'),
 PosixPath('/projects/frame_align/data/news-img-data/2024-04-01_2024-04-30')]

Count number of jpg files in each directory

In [58]:
from tqdm import tqdm

count_monthly_img = {}
for idx, month_dir in enumerate(sorted_month_dir_paths):
    # dir_path = month_dir / "correct_downloaded_imgs"
    # count the number of images in the month directory and its subdirectories
    count_monthly_img[month_dir.name] = len(list(month_dir.glob('*.jpg')))
    print(f"{month_dir.name}:  Downloaded: {count_monthly_img[month_dir.name]}")

2023-05-01_2023-05-31:  Downloaded: 73717
2023-06-01_2023-06-30:  Downloaded: 94839
2023-07-01_2023-07-31:  Downloaded: 80096
2023-08-01_2023-08-31:  Downloaded: 81810
2023-09-01_2023-09-30:  Downloaded: 70022
2023-10-01_2023-10-31:  Downloaded: 24557
2023-11-01_2023-11-30:  Downloaded: 6
2023-12-01_2023-12-31:  Downloaded: 14555
2024-01-01_2024-01-31:  Downloaded: 13282
2024-02-01_2024-02-29:  Downloaded: 14744
2024-03-01_2024-03-31:  Downloaded: 18983
2024-04-01_2024-04-30:  Downloaded: 16735


Make pickle file of the count of jpg files in each directory

In [59]:
sorted_month_dir_paths[0]
# give me base path of this month
sorted_month_dir_paths[0].parent

PosixPath('/projects/frame_align/data/news-img-data')

In [60]:
# make pickle file which is dict of month and uuids in sorted_month_dir_paths/correct_downloaded_imgs

for idx, month_dir in enumerate(tqdm(sorted_month_dir_paths)):
    dir_path = month_dir 
    uuids = [img_path.stem for img_path in dir_path.glob("*.jpg")]
    # print len of duplicated uuids
    print(f"len of duplicated uuids: {len(uuids) - len(set(uuids))}")
    month = month_dir.name
    pickle_path = dir_path.parent / f"{month}_downloaded_uuids.pkl"
    # delete if pickle file already exists
    if pickle_path.exists():
        print(f"pickle file already exists: {pickle_path}, deleting")
        pickle_path.unlink()
    with open(pickle_path, "wb") as f:
        pickle.dump(uuids, f)
        print(f"saved {month} uuids to {pickle_path}") 

  0%|          | 0/12 [00:00<?, ?it/s]

len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2023-05-01_2023-05-31_downloaded_uuids.pkl, deleting


  8%|▊         | 1/12 [00:00<00:05,  1.91it/s]

saved 2023-05-01_2023-05-31 uuids to /projects/frame_align/data/news-img-data/2023-05-01_2023-05-31_downloaded_uuids.pkl
len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2023-06-01_2023-06-30_downloaded_uuids.pkl, deleting


 17%|█▋        | 2/12 [00:01<00:07,  1.36it/s]

...

 67%|██████▋   | 8/12 [00:03<00:00,  4.65it/s]

len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2023-11-01_2023-11-30_downloaded_uuids.pkl, deleting
saved 2023-11-01_2023-11-30 uuids to /projects/frame_align/data/news-img-data/2023-11-01_2023-11-30_downloaded_uuids.pkl
len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2023-12-01_2023-12-31_downloaded_uuids.pkl, deleting
saved 2023-12-01_2023-12-31 uuids to /projects/frame_align/data/news-img-data/2023-12-01_2023-12-31_downloaded_uuids.pkl
len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2024-01-01_2024-01-31_downloaded_uuids.pkl, deleting
saved 2024-01-01_2024-01-31 uuids to /projects/frame_align/data/news-img-data/2024-01-01_2024-01-31_downloaded_uuids.pkl


 83%|████████▎ | 10/12 [00:03<00:00,  6.40it/s]

len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2024-02-01_2024-02-29_downloaded_uuids.pkl, deleting
saved 2024-02-01_2024-02-29 uuids to /projects/frame_align/data/news-img-data/2024-02-01_2024-02-29_downloaded_uuids.pkl
len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2024-03-01_2024-03-31_downloaded_uuids.pkl, deleting
saved 2024-03-01_2024-03-31 uuids to /projects/frame_align/data/news-img-data/2024-03-01_2024-03-31_downloaded_uuids.pkl
len of duplicated uuids: 0
pickle file already exists: /projects/frame_align/data/news-img-data/2024-04-01_2024-04-30_downloaded_uuids.pkl, deleting
saved 2024-04-01_2024-04-30 uuids to /projects/frame_align/data/news-img-data/2024-04-01_2024-04-30_downloaded_uuids.pkl


100%|██████████| 12/12 [00:03<00:00,  3.44it/s]


[Used for Verification] Load pickle file and check the count of jpg files in each directory

In [61]:
# load and count all pickle files
count_monthly_img = {}
for idx, month_dir in enumerate(sorted_month_dir_paths):
    pickle_path = month_dir.parent / f"{month_dir.name}_downloaded_uuids.pkl"
    with open(pickle_path, "rb") as f:
        img_names = pickle.load(f)
        count_monthly_img[month_dir.name] = len(img_names)
        print(f"{month_dir.name}: {count_monthly_img[month_dir.name]}")


2023-05-01_2023-05-31: 73717
2023-06-01_2023-06-30: 94839
2023-07-01_2023-07-31: 80096
2023-08-01_2023-08-31: 81810
2023-09-01_2023-09-30: 70022
2023-10-01_2023-10-31: 24557
2023-11-01_2023-11-30: 6
2023-12-01_2023-12-31: 14555
2024-01-01_2024-01-31: 13282
2024-02-01_2024-02-29: 14744
2024-03-01_2024-03-31: 18983
2024-04-01_2024-04-30: 16735


In [29]:
import os, pickle
import pandas as pd

data_csv_path = f"/projects/frame_align/data/raw/2023-2024/"
data="new_data"
# dir_name ="2023-10-01_2023-10-31"

if data=="old_data":
    og_img_path = f"/projects/frame_align/data/img_data/"
if data=="new_data":
    og_img_path = f"/projects/frame_align/data/news-img-data/"

print(f"Image path: {og_img_path}")



if data=="old_data":
    pkl_path = os.path.join(og_img_path, dir_name, f"{dir_name}_downloaded_uuids.pkl")
if data=="new_data":
    pkl_path = os.path.join(og_img_path, f"{dir_name}_downloaded_uuids.pkl")
with open(pkl_path, "rb") as f:
    downloaded_uuids = pickle.load(f)

# print(f"Downloaded UUIDs: {len(downloaded_uuids)}")


data_csv = os.path.join(data_csv_path, dir_name, "datawithtopiclabels.csv")
df = pd.read_csv(data_csv)

In [13]:
# expand columns
pd.set_option('display.max_colwidth', None)
newdf = data_df[["id", "image_url"]]
newdf.tail()

,id,image_url
62684,d1f9816a-74ac-429b-8b47-cd8c3084ae12,https://img.huffingtonpost.com/asset/6523f63f230000190038c5f1.jpg?cache=ITIXh4onJD&ops=1200_630
62685,75a6dc6d-e1d0-45d8-80b8-e808a8a174f9,https://static.foxnews.com/foxnews.com/content/uploads/2021/11/Rashida-Tlaib-2.jpg
62686,72d69c7d-4c19-4cd7-9d50-24db87767701,https://static.foxnews.com/foxnews.com/content/uploads/2023/10/New-York-Times-Israel-covers.png
62687,e7f2f8b9-7f7a-40ab-b4a6-4c0c86649346,https://a57.foxnews.com/static.foxbusiness.com/foxbusiness.com/content/uploads/2022/04/0/0/Stuart-Varney-4-13-22.jpg?ve=1&tl=1
62688,ba597498-42ac-497b-b7b8-31c4cb30e5ac,https://www.newsmax.com/CMSPages/GetFile.aspx?guid=dfcaf8f7-60da-4105-8fb6-db7e4b8d107f&SiteName=Newsmax


In [ ]:
# delete directory:news-img-data
import shutil
shutil.rmtree("/projects/frame_align/data/news-img-data")


In [ ]:
raw_month_dir = Path("/projects/frame_align/data/raw/2023-2024")
img_dir = month_raw_dir / "img_data/"

In [3]:
file_sizes = {}
for img_file in img_dir.glob("*.jpg"):
    file_sizes[img_file.name.split(".")[0]] = img_file.stat().st_size
file_size_df = pd.DataFrame.from_dict(file_sizes, orient='index', columns=['file_size'])

In [4]:
file_size_df = file_size_df[file_size_df['file_size'] <= file_size_df['file_size'].quantile(0.95)]
file_size_df = file_size_df[file_size_df['file_size'] >= 5000]
# file_size_df.sort_values('file_size', ascending=True).hist(bins=5)

In [5]:
img_uuids = list(file_size_df.index)

In [6]:
len(img_uuids)

36601

In [7]:
text_df = pd.read_csv(month_dir/"datawithtopiclabels.csv")

In [8]:
text_df.columns

Index(['topic', 'auto_topic_label', 'id', 'authors', 'date_download',
       'date_modify', 'date_publish', 'description', 'filename', 'image_url',
       'language', 'localpath', 'maintext', 'source_domain', 'title',
       'title_page', 'title_rss', 'url', 'month', 'topic_label'],
      dtype='object')

In [9]:
text_df.dropna(subset=['maintext'], inplace=True)
text_df['text_length'] = text_df['maintext'].apply(lambda x: len(x))

In [10]:
lower_quantile = text_df['text_length'].quantile(0.05)
upper_quantile = text_df['text_length'].quantile(0.95)

text_df = text_df[text_df['text_length'] >= lower_quantile]
text_df = text_df[text_df['text_length'] <= upper_quantile]

In [17]:
filtered_text_df = []
for topic_label, topic_df in text_df.groupby('topic_label'):
    if len(topic_df) > 5000:
        topic_df = topic_df.sample(5000)
    filtered_text_df.append(topic_df)
filtered_text_df = pd.concat(filtered_text_df)

In [19]:
len(filtered_text_df)

17084

In [20]:
text_uuids = list(filtered_text_df.id)

In [21]:
common_uuids = set(text_uuids).intersection(set(img_uuids))

In [22]:
len(common_uuids)

16140

In [23]:
pickle.dump(common_uuids, open(month_dir/"selected_uuids.pkl", "wb"))